<a href="https://colab.research.google.com/github/klajosw/python/blob/master/kl_py_xls_rip_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left"> 
    <img src="https://raw.githubusercontent.com/klajosw/python/master/kl_mie_python_logo_250.jpg" 
         align="left" width="251" height="251">
    
</p>


<p> </p>

#  Riportok Excelbe mentése és diagram készítés 

##  Pandas és cx_Oracle csomag használatával

https://klajosw.blogspot.com/

https://github.com/klajosw/

---

In [ ]:
## diagram
import cx_Oracle
import pandas as pd
import datetime
us_nev = 'klajos'
us_jsz = 'Ljs'
us_db  = 'aix'

dt = datetime.datetime.now() - datetime.timedelta(days=1)  ## tegnapi nap
datum  = '{:%Y-%m-%d}'.format(dt)

conn = cx_Oracle.connect(us_nev ,us_jsz, us_db)

### Excel filebe mentes
# Excel írható objektum létrehozása (név tartalmazza a futás dátumot)
writer = pd.ExcelWriter(r'c:\Users\kl\Documents\xls\kiv_xml_riport_'+datum+'_kesz.xlsx')

#------------------- riport eredmény lekérdezése és excel mumklaba helyezés kezdet
#--- riport1
sql_com = """
select to_char(END_DATE,'YYYYMMDD') as END_DATE, 
count(distinct XML_FILENAME) as XML_DB, count( distinct INTERNAL_KEY) as szl_db1, count(0) as SZL_DB0 
from DB_STMT_STAT where END_DATE >=  sysdate - 100 
group by END_DATE 
order by END_DATE """


## lekérdezés futtatás
df_ora = pd.read_sql(sql_com, conn)

# Pandas dataframe excelbe írása
df_ora.to_excel(writer, sheet_name='Stat_XML', engine='xlsxwriter', index=False)

workbook  = writer.book
worksheet = writer.sheets['Stat_XML']
(max_row, max_col) = df_ora.shape

# auto szűrő
worksheet.autofilter(0, 0, max_row, max_col-1)

# oszlop szélesítése : pld. az első oszlop 25-re állítása
writer.sheets['Stat_XML'].set_column(0, 0, 25)

# diagram beszúrás
chart = workbook.add_chart({'type': 'line'})               ## column, pie, line, area, scatter' // circle', 'size': 7}
#chart.add_series({'values': ['Stat_XML', 0, 0, 2, 3 ]})
#chart.add_series({'values': '=Stat_XML!$A$1:$C$8'})
#chart.add_series({'values': ['Stat_XML', 1, 1, max_row, 1]})
"""
----------------1.
chart.add_series({
'name': ['Stat_XML', 0, 2],
'categories': ['Stat_XML', 1, 0, max_row, 0],  ## x tengely
'values'    : ['Stat_XML', 1, 2, max_row, 2],  ## y tengely
## 'values'    : ['Stat_XML', 1, 3, max_row, 3],
})
chart.add_series({
'name': ['Stat_XML', 0, 3],
'categories': ['Stat_XML', 1, 0, max_row, 0],  ## x tengely
'values'    : ['Stat_XML', 1, 3, max_row, 3],  ## y tengely
## 'values'    : ['Stat_XML', 1, 3, max_row, 3],
})
--------------- 2.
chart.add_series({
'name': ['Stat_XML', 0, max_col - 1],
'categories': ['Stat_XML', 1, 0, max_row , 0],
'values': ['Stat_XML', 1, max_col - 1, max_row , max_col - 1],
#'overlap':-10,
})
chart.add_series({
'name': ['Stat_XML', 0, max_col - 2],
'categories': ['Stat_XML', 1, 0, max_row , 0],
'values': ['Stat_XML', 1, max_col - 2, max_row , max_col - 2],
#'overlap':-10,
})
chart.add_series({
'name': ['Stat_XML', 0, 1],
'categories': ['Stat_XML', 1, 0, max_row , 0],
'values': ['Stat_XML', 1, 1, max_row , 1],
#'overlap':-10,
})

"""
## első oszlop a kategoria, tobbi oszlop adatok és az értékek 
for i in range(max_col):
    if i != 0 :
        chart.add_series({
'name': ['Stat_XML', 0, i],
'categories': ['Stat_XML', 1, 0, max_row , 0],
'values': ['Stat_XML', 1, i, max_row ,i],
#'overlap':-10,
})



worksheet.insert_chart(1, max_col +1, chart)

#------------------- riport eredmény lekérdezése és excel mumklaba helyezés vége

# Az excel állomány mentése és lezárás
writer.save()
writer.close()
conn.close()
